In [ ]:
# import libraries
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_dataset = pd.read_csv(
    train_file_path,
    sep='\t',
    header=None,
    names=['label', 'message']
)

test_dataset = pd.read_csv(
    test_file_path,
    sep='\t',
    header=None,
    names=['label', 'message']
)

In [ ]:
train_dataset.dropna(inplace=True)
train_dataset.drop_duplicates(inplace=True)

test_dataset.dropna(inplace=True)
test_dataset.drop_duplicates(inplace=True)

In [ ]:
train_dataset.info()

In [ ]:
test_dataset.info()

In [ ]:
train_dataset['label'] = train_dataset['label'].map(lambda x: 1 if x == 'spam' else 0)
test_dataset['label'] = test_dataset['label'].map(lambda x: 1 if x == 'spam' else 0)

In [ ]:
train_dataset.info()

In [ ]:
test_dataset.info()

In [ ]:
train = tf.data.Dataset.from_tensor_slices((train_dataset['message'].values, train_dataset['label'].values))
test = tf.data.Dataset.from_tensor_slices((test_dataset['message'].values, test_dataset['label'].values))

In [ ]:
VOCAB_SIZE = 1000
SHUFFLE = 10000
BATCH = 64

In [ ]:
train = train.shuffle(SHUFFLE).batch(BATCH).prefetch(tf.data.AUTOTUNE)
test = test.batch(BATCH).prefetch(tf.data.AUTOTUNE)

In [ ]:
vectorizer = keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)

vectorizer.adapt(train.map(lambda message, label: message))

In [ ]:
model = keras.Sequential([
    vectorizer,
    keras.layers.Embedding(VOCAB_SIZE, 64, mask_zero=True),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1)
])

model.compile(
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy']
)

In [ ]:
model.fit(
    train,
    epochs=5,
    validation_data=test
)

In [ ]:
# function to predict messages based on model
def predict_message(pred_text):
  prediction = model.predict(tf.constant([pred_text]), batch_size=BATCH)

  if prediction[0][0] > 0:
    prediction = [prediction[0][0], 'spam']
  else:
    prediction = [prediction[0][0], 'ham']

  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()